In [4]:
pip install matplotlib seaborn pandas plotly

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 294 kB 4.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

total_trade = pd.read_csv('datasets/socio-economic/Total_Trade.csv')
annual_value = pd.read_csv('datasets/socio-economic/AnnualValue.csv')
countrywise = pd.read_csv('datasets/socio-economic/Countrywise_Trade_Analysis_1988-2021.csv')

total_trade.head()


,ReporterISO3,ReporterName,PartnerISO3,TotalTradeValue
0,EUN,European Union,WLD,3.898848e+10
1,CHN,China,WLD,3.290890e+10
2,DEU,Germany,WLD,3.205585e+10
3,USA,United States,WLD,2.964380e+10
4,JPN,Japan,WLD,1.858069e+10


In [7]:
annual_value.head()

,ReporterISO3,ReporterName,Year,AnnualTradeValue
0,ABW,Aruba,2000,345998.182
1,ABW,Aruba,2001,297712.283
2,ABW,Aruba,2002,256517.725
3,ABW,Aruba,2003,164381.003
4,ABW,Aruba,2004,159364.052


In [8]:
countrywise.head()

,CountryName,Year,CurrentYearTotalTradeValue,PreviousYearTotalTradeValue,GrowthPercentage,AverageGrowthPercentage
0,Afghanistan,2009,806882.012,1080131.188,-25.30,5.51
1,Afghanistan,2010,776967.274,806882.012,-3.71,5.51
2,Afghanistan,2011,751701.870,776967.274,-3.25,5.51
3,Afghanistan,2012,857805.420,751701.870,14.12,5.51
4,Afghanistan,2013,1029945.966,857805.420,20.07,5.51


In [11]:
def calculate_fseci(annual_df, total_trade_df, countrywise_df):

    economic_power = annual_df.groupby('ReporterISO3').agg({
        'AnnualTradeValue': ['mean', 'max', 'std']
    }).reset_index()
    economic_power.columns = ['CountryCode', 'AvgTradeValue', 'MaxTradeValue', 'TradeStdDev']

    #Normalizing
    economic_power['PurchasingPowerScore'] = (
        (economic_power['AvgTradeValue'] - economic_power['AvgTradeValue'].min()) /
        (economic_power['AvgTradeValue'].max() - economic_power['AvgTradeValue'].min()) * 100
    )

    economic_power['CoeffVariation'] = (
        economic_power['TradeStdDev'] / economic_power['AvgTradeValue']
    )
    economic_power['StabilityScore'] = (
        100 - (economic_power['CoeffVariation'] / economic_power['CoeffVariation'].max() * 100)
    )
    economic_power['StabilityScore'] = economic_power['StabilityScore'].clip(0, 100)

    food_dependency = total_trade[total_trade['PartnerISO3'] == 'WLD'].copy()
    food_dependency = food_dependency.groupby('ReporterISO3')['TotalTradeValue'].sum().reset_index()
    food_dependency.columns = ['CountryCode', 'WorldTrade']

    food_dependency['ImportDependencyRatio'] = (
        food_dependency['WorldTrade'] / food_dependency['WorldTrade'].max()
    )
    food_dependency['FoodIndependenceScore'] = (
        (1 - food_dependency['ImportDependencyRatio']) * 100
    )

    resilience = countrywise_df.groupby('CountryName').agg({
        'GrowthPercentage': ['mean', 'std'],
        'CurrentYearTotalTradeValue': 'mean'
    }).reset_index()
    resilience.columns = ['CountryName', 'AvgGrowth', 'GrowthVolatility', 'AvgCurrentTrade']

    resilience['CrisisResilienceScore'] = (
        100 - (resilience['GrowthVolatility'] / resilience['GrowthVolatility'].max() * 100)
    )

    country_mapping = annual_df[['ReporterISO3', 'ReporterName']].drop_duplicates()
    resilience = resilience.merge(
        country_mapping,
        left_on='CountryName',
        right_on='ReporterName',
        how='left'
    )

    fseci = economic_power.merge(food_dependency, on='CountryCode', how='outer')
    fseci = fseci.merge(
        resilience[['ReporterISO3', 'CrisisResilienceScore', 'AvgGrowth']],
        left_on='CountryCode',
        right_on='ReporterISO3',
        how='left'
    )

    fseci = fseci.fillna({
        'PurchasingPowerScore': 0,
        'StabilityScore': 50,
        'FoodIndependenceScore': 50,
        'CrisisResilienceScore': 50,
        'AvgGrowth': 0
    })

    fseci['FSECI'] = (
        0.35 * fseci['PurchasingPowerScore'] +     
        0.25 * fseci['StabilityScore'] +            
        0.20 * fseci['FoodIndependenceScore'] +     
        0.20 * fseci['CrisisResilienceScore']       
    )

    fseci['VulnerabilityCategory'] = pd.cut(
        fseci['FSECI'],
        bins=[0, 25, 50, 75, 100],
        labels=['Critical', 'High', 'Moderate', 'Low']
    )

    return fseci

fseci_data = calculate_fseci(annual_value, total_trade, countrywise)

country_names = annual_value[['ReporterISO3', 'ReporterName']].drop_duplicates()
fseci_data = fseci_data.merge(
    country_names,
    left_on='CountryCode',
    right_on='ReporterISO3',
    how='left'
)

print("✓ Food Security Economic Capacity Index (FSECI) calculated for", len(fseci_data), "countries")

fig1 = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Distribution of FSECI Scores',
        'Component Scores Comparison',
        'FSECI by Vulnerability Category',
        'Economic Capacity vs Food Independence'
    ),
    specs=[
        [{"type": "histogram"}, {"type": "box"}],
        [{"type": "bar"}, {"type": "scatter"}]
    ]
)

fig1.add_trace(
    go.Histogram(
        x=fseci_data['FSECI'],
        nbinsx=40,
        marker_color='teal',
        name='FSECI',
        showlegend=False
    ),
    row=1, col=1
)

components = ['PurchasingPowerScore', 'StabilityScore', 'FoodIndependenceScore', 'CrisisResilienceScore']
for comp in components:
    fig1.add_trace(
        go.Box(
            y=fseci_data[comp],
            name=comp.replace('Score', ''),
            marker_color=['steelblue', 'orange', 'green', 'purple'][components.index(comp)]
        ),
        row=1, col=2
    )

category_counts = fseci_data['VulnerabilityCategory'].value_counts().sort_index()

fig2 = px.choropleth(
    fseci_data,
    locations='CountryCode',
    color='FSECI',
    hover_name='ReporterName',
    hover_data={
        'CountryCode': False,
        'FSECI': ':.2f',
        'PurchasingPowerScore': ':.1f',
        'StabilityScore': ':.1f',
        'FoodIndependenceScore': ':.1f',
        'VulnerabilityCategory': True
    },
    color_continuous_scale='RdYlGn',
    range_color=[0, 100],
    labels={
        'FSECI': 'Food Security Economic Capacity',
        'PurchasingPowerScore': 'Economic Power',
        'StabilityScore': 'Economic Stability',
        'FoodIndependenceScore': 'Food Independence',
        'VulnerabilityCategory': 'Risk Level'
    },
    title='<b>Global Food Security Economic Capacity Index (FSECI)</b><br>' +
          '<sub>Measures economic ability to maintain food access during climate shocks | Lower scores = higher vulnerability</sub>'
)

fig2.update_layout(
    height=600,
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth',
        bgcolor='rgba(240,240,240,0.9)'
    )
)

fig2.show()


def calculate_yearly_fseci(annual_df):
    yearly_fseci = []

    for year in annual_df['Year'].unique():
        year_data = annual_df[annual_df['Year'] == year]

        # Simple year-specific FSECI
        year_scores = year_data.groupby('ReporterISO3').agg({
            'AnnualTradeValue': 'sum'
        }).reset_index()

        year_scores['FSECI_Proxy'] = (
            (year_scores['AnnualTradeValue'] - year_scores['AnnualTradeValue'].min()) /
            (year_scores['AnnualTradeValue'].max() - year_scores['AnnualTradeValue'].min()) * 100
        )

        yearly_fseci.append({
            'Year': year,
            'AvgFSECI': year_scores['FSECI_Proxy'].mean(),
            'StdFSECI': year_scores['FSECI_Proxy'].std(),
            'NumCountries': len(year_scores)
        })

    return pd.DataFrame(yearly_fseci)

yearly_fseci = calculate_yearly_fseci(annual_value)

fig3 = go.Figure()

fig3.add_trace(go.Scatter(
    x=yearly_fseci['Year'],
    y=yearly_fseci['AvgFSECI'],
    mode='lines+markers',
    name='Average FSECI',
    line=dict(width=3, color='teal'),
    marker=dict(size=8)
))

fig3.add_trace(go.Scatter(
    x=yearly_fseci['Year'],
    y=yearly_fseci['AvgFSECI'] + yearly_fseci['StdFSECI'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

fig3.add_trace(go.Scatter(
    x=yearly_fseci['Year'],
    y=yearly_fseci['AvgFSECI'] - yearly_fseci['StdFSECI'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(0,128,128,0.2)',
    fill='tonexty',
    name='±1 Std Dev',
    hoverinfo='skip'
))

fig3.add_vrect(x0=2008, x1=2009, fillcolor="red", opacity=0.2,
               annotation_text="2008 Crisis", annotation_position="top left")
fig3.add_vrect(x0=2020, x1=2021, fillcolor="orange", opacity=0.2,
               annotation_text="COVID-19", annotation_position="top left")

fig3.update_layout(
    title='<b>Global Food Security Economic Capacity: Temporal Evolution</b><br>' +
          '<sub>Shows widening inequality and vulnerability during crises</sub>',
    xaxis_title='Year',
    yaxis_title='Average FSECI Score',
    height=500,
    hovermode='x unified',
    showlegend=True
)

fig3.show()


top15 = fseci_data.nlargest(15, 'FSECI')[['ReporterName', 'FSECI', 'VulnerabilityCategory']]
bottom15 = fseci_data.nsmallest(15, 'FSECI')[['ReporterName', 'FSECI', 'VulnerabilityCategory']]

fig4 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Top 15 Most Food-Secure (Economically)', 'Top 15 Most Vulnerable'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

fig4.add_trace(
    go.Bar(
        y=top15['ReporterName'],
        x=top15['FSECI'],
        orientation='h',
        marker_color='green',
        text=top15['FSECI'].round(1),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=1
)

fig4.add_trace(
    go.Bar(
        y=bottom15['ReporterName'],
        x=bottom15['FSECI'],
        orientation='h',
        marker_color='darkred',
        text=bottom15['FSECI'].round(1),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

fig4.update_xaxes(title_text="FSECI Score", row=1, col=1)
fig4.update_xaxes(title_text="FSECI Score", row=1, col=2)

fig4.update_layout(
    height=600,
    title_text="<b>Economic Food Security: Best vs Worst Performers</b>",
    showlegend=False
)

fig4.show()



summary = pd.DataFrame({
    'Metric': [
        'Total Countries Analyzed',
        'Average FSECI Score',
        'Countries in Critical Category',
        'Countries in Low Vulnerability',
        'Highest FSECI Country',
        'Lowest FSECI Country',
        'FSECI Standard Deviation',
        'Inequality Coefficient (Gini-like)'
    ],
    'Value': [
        len(fseci_data),
        f"{fseci_data['FSECI'].mean():.2f}",
        len(fseci_data[fseci_data['VulnerabilityCategory'] == 'Critical']),
        len(fseci_data[fseci_data['VulnerabilityCategory'] == 'Low']),
        fseci_data.loc[fseci_data['FSECI'].idxmax(), 'ReporterName'],
        fseci_data.loc[fseci_data['FSECI'].idxmin(), 'ReporterName'],
        f"{fseci_data['FSECI'].std():.2f}",
        f"{fseci_data['FSECI'].std() / fseci_data['FSECI'].mean():.3f}"
    ]
})


fseci_export = fseci_data[[
    'CountryCode', 'ReporterName', 'FSECI', 'VulnerabilityCategory',
    'PurchasingPowerScore', 'StabilityScore', 'FoodIndependenceScore',
    'CrisisResilienceScore'
]].copy()

fseci_export.to_csv('FSECI_SocioEconomic_Indicator.csv', index=False)

✓ Food Security Economic Capacity Index (FSECI) calculated for 207 countries


In [16]:
fseci_score = pd.read_csv('FSECI_SocioEconomic_Indicator.csv')

data = fseci_score[['CountryCode', 'FSECI']]
data.to_csv("analyzed_FSECI_by_country.csv", index=False)
data.head()

,CountryCode,FSECI
0,ABW,59.121571
1,AFG,60.522451
2,AGO,58.362607
3,AIA,57.453869
4,ALB,53.191337
